# 3. Performance Attribution
## Active return
In this first exercise, you're going to calculate the active return of a portfolio that's managed against a benchmark. You've learned many ways of how to calculate total return over a period. For this exercise, you'll use the simple mean returns multiplied with the weights to obtain a total return for both the portfolio as well as the benchmark. Available is portfolio data containing weights and asset returns under portfolio_data. Have a look at the data by running portfolio_data.head(10) in the IPython Shell. Good luck!

In [2]:
nombre_archivo = 'large_pf.csv'

In [3]:
import os

while 'DATABASE' not in os.listdir():
    os.chdir('..')

In [4]:
import pandas as pd


portfolio_data = pd.read_csv('DATABASE/{}'.format(nombre_archivo))\
.set_index('Unnamed: 0')

portfolio_data

,mean_return,variance,pf_weights,bm_weights,Security,GICS Sector,GICS Sub Industry
Unnamed: 0,,,,,,,
A,0.146146,0.035194,0.000,0.000,Agilent Technologies Inc,Health Care,Health Care Equipment
AAL,0.444411,0.094328,0.214,0.000,American Airlines Group,Industrials,Airlines
AAP,0.242189,0.029633,0.000,0.000,Advance Auto Parts,Consumer Discretionary,Automotive Retail
AAPL,0.225074,0.027283,0.000,0.000,Apple Inc.,Information Technology,Computer Hardware
ABBV,0.182541,0.029926,0.000,0.000,AbbVie,Health Care,Pharmaceuticals
...,...,...,...,...,...,...,...
YHOO,0.162994,0.036719,0.000,0.000,Yahoo Inc.,Information Technology,Internet Software & Services
YUM,0.162098,0.023774,0.000,0.000,Yum! Brands Inc,Consumer Discretionary,Restaurants
ZBH,0.103081,0.020171,0.000,0.000,Zimmer Biomet Holdings,Health Care,Health Care Equipment


In [5]:
# Check the portfolio weights
print(portfolio_data.pf_weights.sum())

100.0


In [6]:
# Calculate return of the portfolio
total_return_pf = (portfolio_data['pf_weights']*portfolio_data['mean_return']).sum()
total_return_pf

33.134025872593554

Repeat the steps and calculate the total return of the benchmark.



In [7]:
# Calculate return of the benchmark
total_return_bm = (portfolio_data['bm_weights']*portfolio_data['mean_return']).sum()
total_return_bm

10.557642112692943

In [8]:
# Calculate and print the active return
active_return = total_return_pf - total_return_bm
print ("%.2f%%" % active_return)

22.58%


### Industry attribution
In this exercise, you're going to calculate the relative industry position of your portfolio, versus a benchmark. As a portfolio manager you need to understand your portfolio's underweight and overweight positions (or "industry bets"), as they are a large driver of performance, as well as a potential source of risk.

The DataFrame portfolio_data is available, containing details about the industry classification, obtained from the Global Industry Classification System or "GICS", of your portfolio holdings, as well as your portfolio weights and the benchmark weights.

In [9]:
# Print the sum of the bm and pf weights
print (portfolio_data.bm_weights.sum())

100.001


In [10]:
print (portfolio_data.pf_weights.sum())

100.0


Collapse the DataFrame on the GICS Sector using the groupby command, to be able to see performance and weights on a sector level. Make sure to sum the groupby, to get the summed values for each column.



In [11]:
# Group dataframe by GICS sectors 
grouped_df=portfolio_data.groupby('GICS Sector').sum()
grouped_df

,mean_return,variance,pf_weights,bm_weights
GICS Sector,,,,
Consumer Discretionary,14.091950,3.011996,32.426,12.169
Consumer Staples,5.131286,0.639845,24.551,29.727
Energy,3.390600,1.897641,0.000,0.000
Financials,8.701437,1.879873,0.000,2.116
Health Care,10.747583,2.349866,11.964,9.099
Industrials,11.336994,2.240226,14.142,11.677
Information Technology,11.217113,2.651298,14.903,6.210
Materials,3.033814,0.877748,0.000,2.043
Real Estate,3.959113,0.728886,2.014,0.000


Now calculate the active weights of the portfolio by comparing the portfolio weights in the column pf_weights, with the benchmark weights in the column bm_weights. Inspect the active weights.



In [12]:
# Calculate active weights of portfolio
grouped_df['active_weight']=grouped_df['pf_weights']-grouped_df['bm_weights']
print (grouped_df['active_weight'])

GICS Sector
Consumer Discretionary         20.257
Consumer Staples               -5.176
Energy                          0.000
Financials                     -2.116
Health Care                     2.865
Industrials                     2.465
Information Technology          8.693
Materials                      -2.043
Real Estate                     2.014
Telecommunications Services    -1.367
Utilities                     -25.593
Name: active_weight, dtype: float64


## Risk factors
**Size factor**
Stocks with a **lower market value** (mid and small caps) typically **realize higher returns** than those with a higher market value or capitalization (large caps). The size factor measures this **difference in return** between small cap companies relative to large cap companies. Suppose you would take the correlation of the S&P500 returns and the size factor returns. What would you expect to find?

**CORRECT ANSWER: I would expect a negative correlation between the size factor and the S&P500 returns.**

**The S&P500 consist mostly of large cap companies. Since the size factor returns are positive when small cap companies do well relative to large cap, the two should be negatively related to each other.**



### Value factor
In the previous exercise you've looked at the S&P500 exposures, and saw that there was a large consistent exposure to the value factor, but a very fluctuating correlation to momentum.

Let's now check how our portfolio measures up to this, and let's especially focus on value. Available to you is a DataFrame called factor_data containing the factor returns as well as your portfolio returns. Start by inspecting the DataFrame factor_data in the IPython shell using factor_data.head().

In [15]:

nombre_archivo = 'factors_pf_returns.csv'
factor_data = pd.read_csv('DATABASE/{}'.format(nombre_archivo))\
.set_index('date')\
.rename(columns = {'Mkt-RF':'momentum',  'pf_returns'  :'portfolio'})

factor_data

,momentum,SMB,HML,RF,portfolio
date,,,,,
2016-01-04,-1.59,-0.83,0.53,0.000,-1.651745
2016-01-05,0.12,-0.22,0.01,0.000,0.408161
2016-01-06,-1.35,-0.12,0.00,0.000,0.325555
2016-01-07,-2.44,-0.29,0.08,0.000,-0.698259
2016-01-08,-1.11,-0.47,-0.03,0.000,-0.959352
...,...,...,...,...,...
2016-12-23,0.19,0.56,-0.52,0.001,0.438153
2016-12-27,0.27,0.21,0.13,0.001,0.430411
2016-12-28,-0.87,-0.27,0.08,0.001,-0.903525


In [16]:
# Calculate the pairwise correlation
factor_data.corr()

,momentum,SMB,HML,RF,portfolio
momentum,1.000000,0.331250,0.099800,0.087073,0.738586
SMB,0.331250,1.000000,-0.108081,0.076750,0.162303
HML,0.099800,-0.108081,1.000000,-0.049019,-0.237647
RF,0.087073,0.076750,-0.049019,1.000000,0.039383
portfolio,0.738586,0.162303,-0.237647,0.039383,1.000000


Create a new column in factor_data called correlation_value, and assign the rolling correlation between your portfolio and the value factor to it, using a 5 day window. Plot your results.



In [17]:
'''


# Calculate rolling 5 day correlation 
factor_data['correlation_value']=factor_data['portfolio'].rolling(5).corr(factor_data['value'])

# Plot the rolling correlation
factor_data['correlation_value'].plot()
plt.legend()
plt.show()

'''

"\n\n\n# Calculate rolling 5 day correlation \nfactor_data['correlation_value']=factor_data['portfolio'].rolling(5).corr(factor_data['value'])\n\n# Plot the rolling correlation\nfactor_data['correlation_value'].plot()\nplt.legend()\nplt.show()\n\n"

### Momentum factor
In this exercise, you are going to investigate the correlation of the S&P500 returns with 2 factors, momentum and value.

A stock is showing "momentum" if its prior 12-month average of returns is positive. The momentum factor is therefore formed by combining stocks that show consistent positive historic returns. The value factor looks at stocks that are inexpensive relative to some measure of fundamental value. For example price-to-earnings and price-to-book are typically ratios to measure "value". Let's see how our S&P500 returns relate to the returns of these factors.

Available is a DataFrame df containing the returns of the two factors and the S&P500 over time.